<a href="https://colab.research.google.com/github/saieswaramurali/ATM_machine_with_face_verification/blob/main/ATM_WITH_FACE_VERIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
customer_database = []
# think a way to avoid users with same name ;
# try to come up with an idea to use multithreading here since CAO ;
# should implement OTP for verification
# and add face recognition for two step verification

In [ ]:
from prompt_toolkit.application.application import E
'''
username = 'sai'
pin = 1234
balance = 0
user_image = 0
choice = 0
'''

import math


#class for customers
class customer :

  def __init__(self, username, pin, balance = 0) :
    assert int(math.log10(pin) + 1) == 4 , "pin number is not valid"
    #assert len(username) <= 3, "username cannot be empty"
    assert balance >= 0 , "balance cannot be negative"
    self.username = username
    self.pin = pin
    self.balance = balance

#function to display menu

def start_page() :
  print("""
    WELCOME TO OUR BANK
    ENTER 1 TO LOGIN TO AN EXISTING ACCOUNT
    ENTER 2 TO CREATE AN ACCOUNT
  """)

def Menu() :

  # print("Welcome to our bank")
    print("""
        WELCOME TO OUR BANK
        ****MENU****
        Enter 1 to check balance
        Enter 2 to withdraw the cash
        Enter 3 to deposit the cash
        Enter 4 to cancel
    """)
    choice = int(input())
    while(type(choice) != type(0)) :
      choice = int(input())
    name = str(input("Enter the username : "))
    pin = int(input("Enter the pin : "))

    database_index = None
    # to check whether the user is already exist in the database ;
    flag = 0
    for i in range(len(customer_database)) :
      if customer_database[i].username == name :
        flag = 1
        database_index = i
        break
    if flag == 0 :
      print("!!! KINDLY CHECK THE USERNAME ONCE AGAIN!!!")

    if flag == 1 : #user exists  ;


          if customer_database[database_index].pin == pin :
          # implement multiple verifications like OTP and face verification by adding if statements here ;

                if choice == 4 : # to cancel
                  pass
                elif choice == 3 : # to deposit the cash
                  deposit = int(input("Enter the amount to deposit : "))
                  while deposit <= 0 :
                    print("Enter appropriate amount to deposit")
                    deposit = int(input("Enter the amount to deposit : "))

                  customer_database[database_index].balance += deposit
                  print(f"HENCE the updated balance of account is : {customer_database[database_index].balance}")

                elif choice == 2 : # to withdraw the cash
                  withdraw = int(input("Enter the amount to withdraw : "))

                  while withdraw <= 0 or customer_database[database_index].balance < withdraw : #invalid amount to withdraw
                    print("invalid balance or invalid amount entered")
                    withdraw = int(input("Enter the amount to withdraw : "))

                  customer_database[database_index].balance -= withdraw
                  print(f"HENCE the updated balance of account is {customer_database[database_index].balance}")

                elif choice == 1 : # to check the balance
                  print(f"Your account balance is {customer_database[database_index].balance}")

          else :
            print("!!! kindly check your pin number !!!")






#function to initiate an account for new users
def create_account(username, pin, balance) :
    assert int(math.log10(pin) + 1) == 4 , "pin number is not valid"
    #assert len(username) == 0, "username cannot be empty"
    assert balance >= 0 , "balance cannot be negative"
    #assert username in customer_database.username() , "username is taken already"

    customer_database.append(customer(username, pin, balance))


def Main() :
  start_page()
  choice = int(input())

  if choice == 1 :
    #display the information of the existing user :
    Menu()
  elif choice == 2:
    #want to create a new account for user
    name = str(input("Enter the username : "))
    pin = int(input("Enter the pin : "))
    balance = int(input("Enter the balance : "))
    create_account(name, pin, balance)



#sai = customer("sai", 1222, 9000)
#should try to implement a off button :
#while True :

Main()

# to clear the database if we need to ;
def clear_userDatabase() :
  customer_database.clear()




    WELCOME TO OUR BANK 
    ENTER 1 TO LOGIN TO AN EXISTING ACCOUNT
    ENTER 2 TO CREATE AN ACCOUNT 
  
1
 
        WELCOME TO OUR BANK
        ****MENU**** 
        Enter 1 to check balance 
        Enter 2 to withdraw the cash 
        Enter 3 to deposit the cash 
        Enter 4 to cancel 
    
1
Enter the username : saieswara
Enter the pin : 1233
!!! kindly check your pin number !!!


In [ ]:
#think of a way to avoid users with same username ;
for i in customer_database :
  print(i.username)

saieswara
sai
sai 
